# IML 2018 Modeled

In [1]:
dataset_id = "iml_2018_modeled"

In [2]:
!bq --location=US mk --dataset {dataset_id}

Dataset 'sashimi-266523:iml_2018_modeled' successfully created.


## Tables from Original Public Resources Table

In [11]:
%%bigquery
create table iml_2018_modeled.public_resources as
select MID, DISCIPL, COMMONNAME, LEGALNAME, ADSTREET, ADCITY, ADSTATE, 
cast(ADZIP5 as STRING) as ZIPCODE, 
PHONE, WEBURL, 
cast(replace(INCOME15, ' ', '0') as FLOAT64) as INCOME, 
cast(replace(REVENUE15, ' ', '0') as FLOAT64) as REVENUE, 
LONGITUDE, LATITUDE
from `iml_2018_staging.public_resources` 

""


In [66]:
%%bigquery
select count(*) from `iml_2018_modeled.public_resources`  

,f0_
0,7429


In [67]:
%%bigquery
select count(distinct MID) from `iml_2018_modeled.public_resources`  

,f0_
0,7429
